In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler






In [ ]:
df=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/X_train_N1UvY30.csv")  
df_result=pd.read_csv("/Users/aurelientarroux/Desktop/Projet_prog/Projet_2/Data/y_train_or6m3Ta.csv")   
df_base=df.merge(df_result, on='obs_id', how='inner')

In [ ]:
df=df_base

In [ ]:
df['bid_ask_spread']=df['ask']-df['bid']
for k in ['venue','action','side']:
    df_encoded = pd.get_dummies(df[k], prefix=k+' ')
    df=df.merge(df_encoded,left_index=True,right_index=True)
    df.drop(columns=[k],inplace=True)
df.drop(columns=['o_id'],inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_columns = ['price', 'bid', 'ask', 'bid_size', 'ask_size', 'flux','bid_ask_spread']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


In [ ]:
df

In [ ]:
sequence_length = 100 
grouped = df.groupby('obs_id')

sequences = [tf.convert_to_tensor(group.values[:sequence_length], dtype=tf.float32) for _, group in grouped]
dataset = tf.data.Dataset.from_tensor_slices(sequences)

batch_size = 1000
dataset = dataset.batch(batch_size)

for batch in dataset.take(1):
    print(batch.shape)  # Devrait afficher (batch_size, 100, 30)

In [ ]:
grouped.head()

In [49]:
import tensorflow as tf

# Définition des entrées du modèle (forme (100, 30))
inputs = tf.keras.Input(shape=(100, 30))

# Application du GRU
gru_output = tf.keras.layers.GRU(64, return_sequences=False)(inputs)

# Ajout d'une couche dense finale avec softmax pour la classification
dense_output = tf.keras.layers.Dense(24, activation='softmax')(gru_output)

# Création du modèle
model = tf.keras.Model(inputs=inputs, outputs=dense_output)

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [50]:

# Entraînement du modèle sur tes batches de données
model.fit(dataset, labels, epochs=10, batch_size=batch_size)

NameError: name 'labels' is not defined

: 